# US Presidential Election Analysis: Electoral College, Popular Vote, or Both?

## Objective
This notebook contains the first step in a larger effort to analyze historical US presidential election data. It focuses on scraping the electoral college voting results for all modern US presidential elections (i.e. from 1892 to the present) from the [National Archives Website](https://www.archives.gov/electoral-college/results), and then writing the results to a Postgres Database for subsequent analysis. The following steps are implemented in this notebook:
1. [X] Initial Setup: Import Modules, Define Functions, and Set Parameter Values
2. [X] Scrape Electoral College Data from the National Archives Website
    1. Define Set containing All Presidential Election Years
    2. Define Set containing All US "States" that Vote in Presidential Elections (includes Washington DC)
    3. Scrape National Archive Summary web page for Links to each Election Year's Data
    4. Scrape each Election Year's web page to download the two tables containing all Election Data
    5. Parse the Data for All Election Years into a useable, compact format
    6. Validate Accuracy of Parsed Election Data
3. [X] Transform and Validate Parsed Election Data so that it conforms to a Star Schema design pattern, with Candidate and State Dimension Tables and Electoral Vote Fact table:
    1. Spot Check Parsed Data for Individual Election Years
    2. Transform and Validate Candidate Data
    3. Transform and Validate State Data
    4. Transform and Validate Electoral Votes Data
4. [X] Write Election Data Tables to Postgres

**Updated on 1/18/22**: Parse the "Other" Candidates for the 2016 Presidential Election using the data in the Notes section of Table 2. 
- [ ] Include each Candidate that received at least on Electoral Vote to the Candidate Dimension Table
- [ ] Include the Electoral Votes for each of those new Candidates to the Electoral Vote Fact Table

### Notes
- The National Archives website only contains Electoral College results for US Presidential Elections from 1892 to present. Another data source will be scraped to get Popular Vote data for each Presidential Election
- Currently I'm only scraping information regarding the Presidential Candidates and their electoral college vote tallies; however, Vice Presidential results are also available, so I can circle back to include that data if the need arises
- FYI: Any candidate who wins a majority or plurality of the popular vote nationwide has a good chance of winning in the Electoral College, but there are no guarantees: for example the results of 1824, 1876, 1888, 2000, and 2016 elections (see Reference 1 below).

### Data Sources
1. US Presidential Election Electoral College voting results: https://www.archives.gov/electoral-college/results
2. US States shapefile: https://www2.census.gov/geo/tiger/TIGER2019/STATE/tl_2019_us_state.zip

### References
1. Electoral College History: https://www.archives.gov/electoral-college/history
2. Links to Resources: https://www.archives.gov/electoral-college/links
3. https://towardsdatascience.com/scraping-table-data-from-websites-using-a-single-line-in-python-ba898d54e2bc
4. https://searchdatamanagement.techtarget.com/definition/star-schema

## 1. Setup

### 1.1 Import Modules

In [1]:
# import modules
from bs4 import BeautifulSoup
from db_tools import DBC
import geopandas as gpd
import getpass
import matplotlib.pyplot as plt
import pandas as pd
import requests

### 1.2 Define Functions

In [2]:
def get_html_tables(url, div_id="main-col", find_all=False):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    div = soup.find("div", id=div_id)
    if find_all:
        return div.find_all("table")
    else:
        return div.find("table")
    
def scrape_election_links(archive_url_domain, archive_url_base):
    link_table = get_html_tables(archive_url_domain+archive_url_base)
    return [archive_url_domain+a['href'] for a in link_table.find_all("a")]    

def scrape_raw_election_tables(election_links, us_election_years):
    raw_election_tables = {}
    for link in election_links:
        link_year = int(link.split('/')[-1])
        if link_year in us_election_years:
            raw_election_tables[link_year] = get_html_tables(link, find_all=True)
        else:
            print(f"Error: The link year, {link_year}, parsed from the following link does not match a US election year: \n{link}")
    return raw_election_tables
    
def parse_election_years(data_tables, state_names):
    parsed_years = []
    for ind, year in enumerate(data_tables.keys()):
        print(f"Working on Election Year = {year} ({ind})")
        parsed_dict = parse_election_year_tables(data_tables[year], state_names)
        parsed_dict['year'] = year
        parsed_years.append(parsed_dict)
    return parsed_years

def parse_election_year_tables(year_tables, state_names):
    parsed_tables = {}
    parsed_tables['t1'] = parse_table1(year_tables[0].find_all('tr'))
    parsed_tables['t2'] = parse_table2(year_tables[1].find_all('tr'), state_names)
    return parsed_tables

def parse_table1(t1_rows):
    cp_row_inds = [0, 1]
    cp_row_headers = ["President", "Main Opponent"]
    candidate_party = []
    for ri, rh in zip(cp_row_inds, cp_row_headers):
        candidate_party.append(parse_t1_candidate_party(t1_rows, ri, rh))
    return candidate_party
    
# Parse Table 1 to store the Presidential Candidates Name and Party
def parse_t1_candidate_party(t1_rows, row_ind, row_header):
    if t1_rows[row_ind].find('th').get_text() == row_header:
        row_data = t1_rows[row_ind].find('td').get_text()
        candidate, party = row_data.split(' [')
        return { \
            'president_candidate_name': candidate.strip(" *").replace(",", ""), \
            'president_candidate_party': party.strip(" *]") \
        }
        #return (candidate.strip(" *"), party.strip(" *]"))
    else:
        print(f"Error: Row{row_ind} does not contain data for {row_header}")

def parse_table2(t2_rows, state_names):
    t2_data = {}
    num_candidates = parse_t2_num_candidates(t2_rows[0])
    t2_data['candidate_state'] = parse_t2_candidate_state(t2_rows[1], num_candidates)
    t2_data['votes_by_state'] = parse_t2_votes_by_state(t2_rows[2:], num_candidates, state_names)
    return t2_data

def parse_t2_num_candidates(header_row):
    return int(header_row.find('th', text="For President").get('colspan'))
    
def parse_t2_candidate_state(cs_row, num_candidates):
    cs_cols = cs_row.find_all('td')
    candidate_state = []
    for ci, cs in enumerate(cs_cols[:num_candidates]):
        if cs.find('br'):
            text = " ".join(cs.stripped_strings)
        else:
            text = cs.get_text()
        if text == "Other":
            candidate, state = text, None
        else:
            candidate, state = text.split(" of ")
            candidate = candidate.strip(", *").replace(",", "")
            state = state.strip(" *").replace(",", "")
        candidate_state.append({ \
            'president_candidate_name': candidate, \
            'col_ind': ci+1, \
            'president_candidate_state': state \
        })
        #candidate_state.append((candidate.strip(", *"), state.strip(", *")))
    return candidate_state

def parse_t2_votes_by_state(states_rows, num_candidates, state_names):
    votes_by_state = []
    for sr in states_rows:
        state_cols = sr.find_all('td')
        col_0_text = state_cols[0].get_text().strip(" *")
        if col_0_text in state_names:
            state = col_0_text
            start_ind, end_ind = 1, num_candidates+2
        elif col_0_text in {"Total", "Totals"}:
            state = "Totals"
            start_ind, end_ind = 1, num_candidates+2
        elif sr.find('th', text='Total'):
            state = "Totals"
            start_ind, end_ind = 0, num_candidates+1
        else:
            state = False
        # Only parse vote data and store it if a valid state value is found
        # This helps validate the state name was parsed correctly, and
        # skips the Notes row at the end of some of the tables
        if state:
            state_votes = {'state': state}
            for si, sv in enumerate(state_cols[start_ind:end_ind]):
                votes = sv.get_text()
                if si == 0:
                    si = 'total_electoral_votes'
                state_votes[si] = int(votes) if votes != '-' else 0
            votes_by_state.append(state_votes)
    return votes_by_state

def print_election_year_results(parsed_year):
    print(f"Election Year: {parsed_year['year']}")
    print(f"Table 1 Top 2 Candidates + Party: \n{pprint_list_of_dicts(parsed_year['t1'])}")
    print(f"Table 2 Candidates + Home State: \n{pprint_list_of_dicts(parsed_year['t2']['candidate_state'])}")
    print(f"Table 2 Votes by State: \n{pprint_list_of_dicts(parsed_year['t2']['votes_by_state'])}")
    
def pprint_list_of_dicts(list_of_dicts):
    list_of_strings = [str(d).strip('{}') for d in list_of_dicts]
    return '\t'+'\n\t'.join(list_of_strings)

def get_name_middle_last(middle_last):
    try:
        split = middle_last.split()
    except:
        split = []
    if len(split) == 1:
        return (None, split[0])
    elif len(split) > 1:
    # This assumes any space in the name is part of the last name
        return (split[0], " ".join(split[1:]))
    else:
        return (None, None)

def create_tables_from_dfs(dbc, schema, table_names, table_column_defs, dfs, replace=True, close=False):
    """This function builds a new schema, schema, in a postgres db using the
    input psycopg2 connection class instance, dbc, then builds one or more tables
    with each table's name specified in the list, table_names, using eac table
    table column definition specified in the list, table_column_defs, and finally
    populates eac table with the data from the list of dataframes, dfs.
    """
    print(f"Creating Schema: {schema}")
    dbc.create_schema(schema, replace=replace)
    for t_ind, t_name in enumerate(table_names):
        print(f"Creating Table: {schema}.{t_name}")
        dbc.create_table(schema, t_name, table_column_defs[t_ind], replace=replace)
        dbc.insert_df_into_table(schema, t_name, dfs[t_ind])
    print("All Data Successfully Inserted!!!")
    if close:
        dbc.close_connection()

def make_map_usa(df, col2plot, figsize=(15,8), title=None, fontsize=18, cmap='Blues', edgecolor='k'):
    # Define fixed longitude limit, xlim, and latitude limit, ylim, for USA
    xlim = (-172, -58)
    ylim = (16, 74)
    fig, ax = plt.subplots(1, figsize=figsize)
    df.plot(column=col2plot, ax=ax, cmap=cmap, edgecolor=edgecolor)
    ax.axis('off')
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=df[col2plot].min(), vmax=df[col2plot].max()))
    sm._A = []
    cb = fig.colorbar(sm)
    cb.set_label(col2plot, fontsize=fontsize)
    cb.ax.tick_params(labelsize=fontsize)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    if title:
        ax.set_title(title, fontsize=fontsize+4)
    plt.tight_layout()

### 1.3 Set Parameters

Define the base URL for the National Archives, and the resource location for the summary page containing links to Presidential election data for each year. The shape file containing US State data can be downloaded from [this link](https://www2.census.gov/geo/tiger/TIGER2019/STATE/).

In [3]:
latest_election_year = 2020
archive_url_domain = "https://www.archives.gov"
archive_url_base = "/electoral-college/results"
usa_state_shp = "/home/fdpearce/Documents/Projects/data/Maps/State_Shapes/tl_2019_us_state/tl_2019_us_state.shp"

## 2. Scrape Electoral College Data from the National Archives Website

### 2.1 Define Set Containing All Presidential Election Years

Create a set, us_election_years, with every year that a US Presidential Election occurred. This set will be used to scrape all available election data. See the `archive_url` website for the complete list of election years. A set is used so membership checks execute efficiently (O(1)).

In [4]:
us_election_years = [1789]+list(range(1792, latest_election_year+4, 4))
print(*us_election_years)

1789 1792 1796 1800 1804 1808 1812 1816 1820 1824 1828 1832 1836 1840 1844 1848 1852 1856 1860 1864 1868 1872 1876 1880 1884 1888 1892 1896 1900 1904 1908 1912 1916 1920 1924 1928 1932 1936 1940 1944 1948 1952 1956 1960 1964 1968 1972 1976 1980 1984 1988 1992 1996 2000 2004 2008 2012 2016 2020


In [5]:
us_election_years = set(us_election_years)

### 2.2 Define Set Containing All US States

Load all the data from the USA States shape file. For now, only the state names are extracted, but later the geometry column can be used to generate maps, extract state features, etc. The names of US Territories that don't participate in Presidential Elections are dropped from the variable containing the set of states.

In [6]:
usa = gpd.read_file(usa_state_shp)
us_state_names = usa['NAME'].values
print(sorted(us_state_names))

['Alabama', 'Alaska', 'American Samoa', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Commonwealth of the Northern Mariana Islands', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Guam', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Puerto Rico', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'United States Virgin Islands', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']


In [7]:
us_state_names = set(us_state_names)
print(f"Total # of States = {len(us_state_names)}")

Total # of States = 56


In [8]:
territories = ['American Samoa', 'Commonwealth of the Northern Mariana Islands', 'Guam', 'Puerto Rico', 'United States Virgin Islands']
us_state_names.difference_update(territories)
print(f"Total # of States = {len(us_state_names)}")

Total # of States = 51


### 2.3 Scrape the Links to each Election Year's Data

Parse html summary page and extract links to each page containing data for a given Presidential Election Year.

In [9]:
election_links = scrape_election_links(archive_url_domain, archive_url_base)
print(election_links)

['https://www.archives.gov/electoral-college/1892', 'https://www.archives.gov/electoral-college/1896', 'https://www.archives.gov/electoral-college/1900', 'https://www.archives.gov/electoral-college/1904', 'https://www.archives.gov/electoral-college/1908', 'https://www.archives.gov/electoral-college/1912', 'https://www.archives.gov/electoral-college/1916', 'https://www.archives.gov/electoral-college/1920', 'https://www.archives.gov/electoral-college/1924', 'https://www.archives.gov/electoral-college/1928', 'https://www.archives.gov/electoral-college/1932', 'https://www.archives.gov/electoral-college/1936', 'https://www.archives.gov/electoral-college/1940', 'https://www.archives.gov/electoral-college/1944', 'https://www.archives.gov/electoral-college/1948', 'https://www.archives.gov/electoral-college/1952', 'https://www.archives.gov/electoral-college/1956', 'https://www.archives.gov/electoral-college/1960', 'https://www.archives.gov/electoral-college/1964', 'https://www.archives.gov/elec

### 2.4 Scrape the Two Tables Containing each Election Year's Data

Data tables dict has keys for each year data is available. Each value is a list with html for the two tables containing election data, which are stored in their own variables primarily for debugging purposes.

In [10]:
raw_election_tables = scrape_raw_election_tables(election_links, us_election_years)

In [11]:
print("Data is currently available from the National Archives website for the following election years:")
print(*raw_election_tables.keys(), sep=", ")

Data is currently available from the National Archives website for the following election years:
1892, 1896, 1900, 1904, 1908, 1912, 1916, 1920, 1924, 1928, 1932, 1936, 1940, 1944, 1948, 1952, 1956, 1960, 1964, 1968, 1972, 1976, 1980, 1984, 1988, 1992, 1996, 2000, 2004, 2008, 2012, 2016, 2020


### 2.5 Parse the Data for All Election Years

In [12]:
parsed_election_years = parse_election_years(raw_election_tables, us_state_names)

Working on Election Year = 1892 (0)
Working on Election Year = 1896 (1)
Working on Election Year = 1900 (2)
Working on Election Year = 1904 (3)
Working on Election Year = 1908 (4)
Working on Election Year = 1912 (5)
Working on Election Year = 1916 (6)
Working on Election Year = 1920 (7)
Working on Election Year = 1924 (8)
Working on Election Year = 1928 (9)
Working on Election Year = 1932 (10)
Working on Election Year = 1936 (11)
Working on Election Year = 1940 (12)
Working on Election Year = 1944 (13)
Working on Election Year = 1948 (14)
Working on Election Year = 1952 (15)
Working on Election Year = 1956 (16)
Working on Election Year = 1960 (17)
Working on Election Year = 1964 (18)
Working on Election Year = 1968 (19)
Working on Election Year = 1972 (20)
Working on Election Year = 1976 (21)
Working on Election Year = 1980 (22)
Working on Election Year = 1984 (23)
Working on Election Year = 1988 (24)
Working on Election Year = 1992 (25)
Working on Election Year = 1996 (26)
Working on 

## 3. Transform and Validate Parsed Election Data
The data transformation performed in this section ultimately creates three tables, following a star schema design (see Reference 3 above):
1. Candidate dimension table built in Section 3.2
2. State dimension table built in Section 3.3
3. Votes by Year fact table built in Section 3.4

Data validation is performed in Section 3.1, and in each of the sections outlined above.

### 3.1 Spot Check Parsed Data for Individual Election Years
Print a compact view of the data parsed for a given election year. The year index value is available in the results of the previous cell. This output provides a useful reference of the parsed data structure for a given election year that is used for development of later sections.

In [13]:
year_index = 31
print_election_year_results(parsed_election_years[year_index])

Election Year: 2016
Table 1 Top 2 Candidates + Party: 
	'president_candidate_name': 'Donald J. Trump', 'president_candidate_party': 'R'
	'president_candidate_name': 'Hillary Clinton', 'president_candidate_party': 'D'
Table 2 Candidates + Home State: 
	'president_candidate_name': 'Donald Trump', 'col_ind': 1, 'president_candidate_state': 'New York'
	'president_candidate_name': 'Other', 'col_ind': 2, 'president_candidate_state': None
	'president_candidate_name': 'Hillary Clinton', 'col_ind': 3, 'president_candidate_state': 'New York'
	'president_candidate_name': 'Other', 'col_ind': 4, 'president_candidate_state': None
Table 2 Votes by State: 
	'state': 'Alabama', 'total_electoral_votes': 9, 1: 9, 2: 0, 3: 0, 4: 0
	'state': 'Alaska', 'total_electoral_votes': 3, 1: 3, 2: 0, 3: 0, 4: 0
	'state': 'Arizona', 'total_electoral_votes': 11, 1: 11, 2: 0, 3: 0, 4: 0
	'state': 'Arkansas', 'total_electoral_votes': 6, 1: 6, 2: 0, 3: 0, 4: 0
	'state': 'California', 'total_electoral_votes': 55, 1: 0, 2:

### 3.2 Transform and Validate Candidate Data
This section creates a Candidate fact table that will be written to Postgres in subsequent steps. I start with the Candidate State data from Table 2 as that gives the complete list of presidential candidates that received electoral votes, except for the troubling 2016 election that has values of other, due to the unprecedented number of presidential candidates that received electoral votes. I'd need to circle back to parse the Notes section on Table 2 to get all candidate names that received at least one electoral vote in 2016, and it may simply be easier to enter that info manually at a later date...

#### **3.2.1 Extract, Validate and Transform Table 2 Data**

In [14]:
t2_states_df = pd.json_normalize(parsed_election_years, ['t2', 'candidate_state'], ['year'])

In [15]:
t2_states_df.head()

,president_candidate_name,col_ind,president_candidate_state,year
0,Grover Cleveland,1,New York,1892
1,Benjamin Harrison,2,Indiana,1892
2,James B. Weaver,3,Iowa,1892
3,William McKinley,1,Ohio,1896
4,William J. Bryan,2,Nebraska,1896


In [16]:
t2_states_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   president_candidate_name   79 non-null     object
 1   col_ind                    79 non-null     int64 
 2   president_candidate_state  77 non-null     object
 3   year                       79 non-null     object
dtypes: int64(1), object(3)
memory usage: 2.6+ KB


In [17]:
t2_states_df[t2_states_df['president_candidate_state'].isna()]

,president_candidate_name,col_ind,president_candidate_state,year
74,Other,2,None,2016
76,Other,4,None,2016


#### Add Candidate Names Corresponding to 'name'="Other"

In [18]:
other_candidate_inds = t2_states_df[t2_states_df['president_candidate_state'].isna()].index
t2_states_df[other_candidate_inds[0]-1:]

,president_candidate_name,col_ind,president_candidate_state,year
73,Donald Trump,1,New York,2016
74,Other,2,None,2016
75,Hillary Clinton,3,New York,2016
76,Other,4,None,2016
77,Joseph R. Biden Jr.,1,Delaware,2020
78,Donald J. Trump,2,Florida,2020


In [19]:
# Collected from Notes section here: https://www.archives.gov/electoral-college/2016
# I didn't assign Colin Powell a state. While "New York" is where he grew up, he wasn't a
# politician so there is no easy way to assign a politically-defined state, thus None seems appropriate
# 'col_ind' value for the added candidates can't be 1 or 3, as those identify the primary candidates,
# and the 'col_ind' values are used to join to the votes data in Section 3.4, then they're discarded in
# favor of the electoral vote rank
other_candidates_df = pd.DataFrame({'president_candidate_name': ["Bernie Sanders", "Ron Paul", "John Kasich", "Colin Powell", "Faith Spotted Eagle"],
                                    'col_ind': [2, 4, 5, 6, 7],
                                    'president_candidate_state': ["Vermont", "Texas", "Ohio", None, "South Dakota"],
                                    'year': [2016, 2016, 2016, 2016, 2016]},
                                    index=[76, 74, 74.1, 74.2, 76.1])
other_candidates_df

,president_candidate_name,col_ind,president_candidate_state,year
76.0,Bernie Sanders,2,Vermont,2016
74.0,Ron Paul,4,Texas,2016
74.1,John Kasich,5,Ohio,2016
74.2,Colin Powell,6,None,2016
76.1,Faith Spotted Eagle,7,South Dakota,2016


In [20]:
t2_states_df.drop(index=other_candidate_inds, inplace=True)
t2_states_df = pd.concat([t2_states_df, other_candidates_df], axis=0)
t2_states_df = t2_states_df.sort_index().reset_index(drop=True)
t2_states_df[other_candidate_inds[0]-1:]

,president_candidate_name,col_ind,president_candidate_state,year
73,Donald Trump,1,New York,2016
74,Ron Paul,4,Texas,2016
75,John Kasich,5,Ohio,2016
76,Colin Powell,6,None,2016
77,Hillary Clinton,3,New York,2016
78,Bernie Sanders,2,Vermont,2016
79,Faith Spotted Eagle,7,South Dakota,2016
80,Joseph R. Biden Jr.,1,Delaware,2020
81,Donald J. Trump,2,Florida,2020


#### Build Initial Candidates DataFrame

In [21]:
candidates_df = t2_states_df[['president_candidate_name', 'president_candidate_state']].drop_duplicates().reset_index(drop=True)

In [22]:
num_can_st = len(t2_states_df[['president_candidate_name', 'president_candidate_state']])
print(f"Original number of Candidate-State combinations = {num_can_st}")
num_unique_can_st = len(candidates_df)
print(f"Unique number of Candidate-State combinations = {num_unique_can_st}")
num_unique_can = len(candidates_df.drop_duplicates(subset='president_candidate_name'))
print(f"Unique number of Candidates = {num_unique_can}")
print(f"Unique number of Other Candidates = {len(other_candidates_df)}")

Original number of Candidate-State combinations = 82
Unique number of Candidate-State combinations = 61
Unique number of Candidates = 60
Unique number of Other Candidates = 5


#### Update Column Names

In [23]:
# Remove 'president_candidate_' prefix and change 'name' to 'full_name'
candidates_df.columns = candidates_df.columns.str.replace("president_candidate_", "")

#### Use Full Name to Create Columns for First, Middle, Last, and Suffix of Name

In [24]:
# Take first split on candidate name to get first name (that's the easy one, lol)
candidates_df[['name_first', 'name_remainder']] = candidates_df['name'].str.split(n=1, expand=True)

In [25]:
# Remove Jr. suffix, commas, leading/trailing spaces from name_remainder prior to parsing middle/last name
candidates_df['name_remainder'] = candidates_df['name_remainder'].str.replace(r",? Jr\.?$", "", regex=True)

In [26]:
# Use get_name_middle_last function to split name_remainder into middle and last name
candidates_df[['name_middle', 'name_last']] = pd.DataFrame(candidates_df['name_remainder'].map(get_name_middle_last).tolist(), index=candidates_df.index)
candidates_df.drop('name_remainder', axis=1, inplace=True)

In [27]:
# Finally add column containing suffix "Jr." if present in full name
candidates_df['name_suffix'] = candidates_df['name'].map(lambda x: "Jr." if x.endswith("Jr.") else None)
candidates_df

,name,state,name_first,name_middle,name_last,name_suffix
0,Grover Cleveland,New York,Grover,None,Cleveland,None
1,Benjamin Harrison,Indiana,Benjamin,None,Harrison,None
2,James B. Weaver,Iowa,James,B.,Weaver,None
3,William McKinley,Ohio,William,None,McKinley,None
4,William J. Bryan,Nebraska,William,J.,Bryan,None
...,...,...,...,...,...,...
56,Hillary Clinton,New York,Hillary,None,Clinton,None
57,Bernie Sanders,Vermont,Bernie,None,Sanders,None
58,Faith Spotted Eagle,South Dakota,Faith,Spotted,Eagle,None
59,Joseph R. Biden Jr.,Delaware,Joseph,R.,Biden,Jr.


#### Fix Spotted Eagle split into middle and last name

In [28]:
candidates_df.loc[candidates_df['name']=="Faith Spotted Eagle", 'name_middle'] = None
candidates_df.loc[candidates_df['name']=="Faith Spotted Eagle", 'name_last'] = "Spotted Eagle"
candidates_df[candidates_df['name']=="Faith Spotted Eagle"]

,name,state,name_first,name_middle,name_last,name_suffix
58,Faith Spotted Eagle,South Dakota,Faith,None,Spotted Eagle,None


#### Validate there are no duplicate candidate names
Duplicate candidate names within this context refers to a candidate that has two different full name spellings in two or more election years, but it is the same candidate. As the table above shows
- **The two Donald Trump rows are true duplicates within this context and need to be updated**
- The two George Bush rows are expected as these are the father-son pair
- The two Richard Nixon rows duplicates are due to his two state affiliations, which will be handled later in this section

In [29]:
name_first_last = candidates_df.groupby(['name_first', 'name_last'], as_index=False).size()
name_first_last[name_first_last['size']>1]

,name_first,name_last,size
12,Donald,Trump,2
16,George,Bush,2
42,Richard,Nixon,2


In [30]:
candidates_df[(candidates_df['name_first']=='Donald')&(candidates_df['name_last']=='Trump')]

,name,state,name_first,name_middle,name_last,name_suffix
52,Donald Trump,New York,Donald,None,Trump,None
60,Donald J. Trump,Florida,Donald,J.,Trump,None


#### Correct Donald Trump's full name and middle initial on row for New York

In [31]:
candidates_df.loc[(candidates_df['name_first']=='Donald')&(candidates_df['name_last']=='Trump')&(candidates_df['state']=='New York'), 'name'] = \
candidates_df.loc[(candidates_df['name_first']=='Donald')&(candidates_df['name_last']=='Trump')&(candidates_df['state']=='Florida'), 'name'].values[0]
candidates_df.loc[(candidates_df['name_first']=='Donald')&(candidates_df['name_last']=='Trump')&(candidates_df['state']=='New York'), 'name_middle'] = \
candidates_df.loc[(candidates_df['name_first']=='Donald')&(candidates_df['name_last']=='Trump')&(candidates_df['state']=='Florida'), 'name_middle'].values[0]

In [32]:
candidates_df[(candidates_df['name_first']=='Donald')&(candidates_df['name_last']=='Trump')]

,name,state,name_first,name_middle,name_last,name_suffix
52,Donald J. Trump,New York,Donald,J.,Trump,None
60,Donald J. Trump,Florida,Donald,J.,Trump,None


#### Aggregate Candidates with Multiple State Affiliations
Group by candidate name to obtain a DataFrame at the correct grain, i.e. one row per candidate. For candidates with more than one state affiliation, combine their state affiliations into a single column, 'state', with each state separated by a hyphen. The current approach produces the desired result without the need for presorting the state values because I want the first election year's party affiliation to be the primary one.
- **Richard M. Nixon and Donald J. Trump are the only two candidate with more than one State association**

In [33]:
state_row_count = candidates_df.groupby('name', as_index=False).size()
state_row_count[state_row_count['size']>1]

,name,size
12,Donald J. Trump,2
43,Richard M. Nixon,2


In [34]:
candidates_df[(candidates_df['name']=="Richard M. Nixon")|(candidates_df['name']=="Donald J. Trump")]

,name,state,name_first,name_middle,name_last,name_suffix
27,Richard M. Nixon,California,Richard,M.,Nixon,None
31,Richard M. Nixon,New York,Richard,M.,Nixon,None
52,Donald J. Trump,New York,Donald,J.,Trump,None
60,Donald J. Trump,Florida,Donald,J.,Trump,None


In [35]:
candidates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         61 non-null     object
 1   state        60 non-null     object
 2   name_first   61 non-null     object
 3   name_middle  36 non-null     object
 4   name_last    61 non-null     object
 5   name_suffix  2 non-null      object
dtypes: object(6)
memory usage: 3.0+ KB


In [36]:
candidates_df = candidates_df.groupby(['name', 'name_first', 'name_middle', 'name_last', 'name_suffix'], sort=False, dropna=False)['state'].agg( \
                                                                                                state=lambda x: "-".join(i if i is not None else "" for i in x)).reset_index()
candidates_df

,name,name_first,name_middle,name_last,name_suffix,state
0,Grover Cleveland,Grover,NaN,Cleveland,NaN,New York
1,Benjamin Harrison,Benjamin,NaN,Harrison,NaN,Indiana
2,James B. Weaver,James,B.,Weaver,NaN,Iowa
3,William McKinley,William,NaN,McKinley,NaN,Ohio
4,William J. Bryan,William,J.,Bryan,NaN,Nebraska
5,Theodore Roosevelt,Theodore,NaN,Roosevelt,NaN,New York
6,Alton B. Parker,Alton,B.,Parker,NaN,New York
7,William H. Taft,William,H.,Taft,NaN,Ohio
8,Woodrow Wilson,Woodrow,NaN,Wilson,NaN,New Jersey
9,Charles E. Hughes,Charles,E.,Hughes,NaN,New York


#### Parse State Column to Create Primary, Secondary State Columns

In [37]:
# Take first split on candidate name to get first name (that's the easy one, lol)
candidates_df[['state', 'state_2']] = candidates_df['state'].str.split("-", n=1, expand=True)
candidates_df.loc[candidates_df['state']=="", 'state'] = None
candidates_df

,name,name_first,name_middle,name_last,name_suffix,state,state_2
0,Grover Cleveland,Grover,NaN,Cleveland,NaN,New York,None
1,Benjamin Harrison,Benjamin,NaN,Harrison,NaN,Indiana,None
2,James B. Weaver,James,B.,Weaver,NaN,Iowa,None
3,William McKinley,William,NaN,McKinley,NaN,Ohio,None
4,William J. Bryan,William,J.,Bryan,NaN,Nebraska,None
5,Theodore Roosevelt,Theodore,NaN,Roosevelt,NaN,New York,None
6,Alton B. Parker,Alton,B.,Parker,NaN,New York,None
7,William H. Taft,William,H.,Taft,NaN,Ohio,None
8,Woodrow Wilson,Woodrow,NaN,Wilson,NaN,New Jersey,None
9,Charles E. Hughes,Charles,E.,Hughes,NaN,New York,None


#### **Validate Grain is Correct: One Candidate per Row**

In [38]:
print(f"Q: Does each row correspond to a unique candidate name? A: {len(candidates_df) == len(candidates_df.drop_duplicates(subset='name'))}")

Q: Does each row correspond to a unique candidate name? A: True


#### **Validate State Distributions are Correct**

In [39]:
num_candidates = len(candidates_df)

In [40]:
state_counts = candidates_df['state'].value_counts(dropna=False)
state_counts

New York          11
Ohio               5
Massachusetts      5
Texas              5
California         4
Kansas             2
South Dakota       2
Minnesota          2
Arizona            2
Illinois           2
Alabama            2
Missouri           1
Georgia            1
Vermont            1
NaN                1
North Carolina     1
Tennessee          1
Arkansas           1
Michigan           1
Iowa               1
South Carolina     1
Nebraska           1
New Jersey         1
West Virginia      1
Virginia           1
Indiana            1
Wisconsin          1
Delaware           1
Name: state, dtype: int64

In [41]:
state_2_counts = candidates_df['state_2'].value_counts(dropna=False)
state_2_counts

NaN         57
New York     1
Florida      1
Name: state_2, dtype: int64

#### **3.2.2 Extract, Validate and Transform Table 1 Candidate Data**

In [42]:
t1_df = pd.json_normalize(parsed_election_years, 't1', ['year'])

In [43]:
t1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   president_candidate_name   66 non-null     object
 1   president_candidate_party  66 non-null     object
 2   year                       66 non-null     object
dtypes: object(3)
memory usage: 1.7+ KB


In [44]:
t1_df.head()

,president_candidate_name,president_candidate_party,year
0,Grover Cleveland,D,1892
1,Benjamin Harrison,R,1892
2,William McKinley,R,1896
3,William J. Bryan,D-P,1896
4,William McKinley,R,1900


In [45]:
t1_df['president_candidate_party'].value_counts()

R      32
D      31
D-P     2
P       1
Name: president_candidate_party, dtype: int64

In [46]:
num_can_pa = len(t1_df[['president_candidate_name', 'president_candidate_party']])
print(f"Original number of Candidate-Party combinations = {num_can_pa}")
num_unique_can_pa = len(t1_df.drop_duplicates(subset=['president_candidate_name', 'president_candidate_party']))
print(f"Unique number of Candidate-Party combinations = {num_unique_can_pa}")
num_unique_can = len(t1_df.drop_duplicates(subset='president_candidate_name'))
print(f"Unique number of Candidates = {num_unique_can}")

Original number of Candidate-Party combinations = 66
Unique number of Candidate-Party combinations = 47
Unique number of Candidates = 45


In [47]:
# William J. Bryan has two party affiliations: Primary = "D" and Secondary = "P"
t1_df[t1_df['president_candidate_name']=="William J. Bryan"]

,president_candidate_name,president_candidate_party,year
3,William J. Bryan,D-P,1896
5,William J. Bryan,D-P,1900
9,William J. Bryan,D,1908


In [48]:
# Only William J. Bryan has a split party designation
t1_df[t1_df['president_candidate_party'].str.contains("-")]

,president_candidate_name,president_candidate_party,year
3,William J. Bryan,D-P,1896
5,William J. Bryan,D-P,1900


In [49]:
# Theodore Roosevelt is the only candidate to change parties: Primary = "R" and Secondary = "P"
t1_df[t1_df['president_candidate_name']=='Theodore Roosevelt']

,president_candidate_name,president_candidate_party,year
6,Theodore Roosevelt,R,1904
11,Theodore Roosevelt,P,1912


#### Build Candidates Party DataFrame
This Candidate table will be joined to the one above to add party affiliation if available.

In [50]:
candidates_party_df = t1_df[['president_candidate_name', 'president_candidate_party']].drop_duplicates().reset_index(drop=True)

#### Update Column Names

In [51]:
# Remove 'president_candidate_' prefix and change 'name' to 'full_name'
candidates_party_df.columns = candidates_party_df.columns.str.replace("president_candidate_", "")
candidates_party_df.head()

,name,party
0,Grover Cleveland,D
1,Benjamin Harrison,R
2,William McKinley,R
3,William J. Bryan,D-P
4,Theodore Roosevelt,R


#### Aggregate Candidates with Multiple Party Affiliations
Combine all party affiliations into a single column, 'party', separated by hyphens. This aggregation also yields a DataFrame at the correct grain, i.e. one row per candidate. Note that this produces the desired result without the need for sorting as I want the first party affiliation to be the primary one. May need to revisit this assumption in the future...

In [52]:
candidates_party_df = candidates_party_df.groupby('name')['party'].agg(party="-".join).reset_index()

#### Parse Party Column to Create Primary, Secondary Party Columns

In [53]:
# Take first split on candidate name to get first name (that's the easy one, lol)
candidates_party_df[['party', 'party_2']] = candidates_party_df['party'].str.split("-", n=1, expand=True)
candidates_party_df['party_2'] = candidates_party_df['party_2'].map(lambda x: x[0] if x else None)
candidates_party_df

,name,party,party_2
0,Adlai Stevenson,D,None
1,Albert Gore Jr.,D,None
2,Alfred E. Smith,D,None
3,Alfred M. Landon,R,None
4,Alton B. Parker,D,None
5,Barack Obama,D,None
6,Barry M. Goldwater,R,None
7,Benjamin Harrison,R,None
8,Bob Dole,R,None
9,Calvin Coolidge,R,None


#### Validate Grain is Correct: One Candidate per Row

In [54]:
print(f"Q: Does each row correspond to a unique candidate name? A: {len(candidates_party_df) == len(candidates_party_df.drop_duplicates(subset='name'))}")

Q: Does each row correspond to a unique candidate name? A: True


#### Validate Party Distributions are Correct

In [55]:
party_counts = candidates_party_df['party'].value_counts(dropna=True)
party_counts

D    23
R    22
Name: party, dtype: int64

In [56]:
party_2_counts = candidates_party_df['party_2'].value_counts(dropna=True)
party_2_counts

P    2
Name: party_2, dtype: int64

#### **3.2.3 Check Consistency of Names from Table 1 and Table 2**

Full names that don't match for the same candidate between the two datasets need to be fixed so that the joining of the two datasets works correctly.
- Update 'Bob Dole in Table 1 to 'Robert Dole' so it matches the name value in Table 2
- Update 'George McGovern' in Table 2 to 'George S. McGovern' so it matches the name value in Table 1, and update his 'name_middle' value to 'S. in Table 2

In [57]:
t1_names = set(candidates_party_df['name'].unique())
t1_t2_name_diffs = t1_names.difference(set(candidates_df['name'].unique()))
print(t1_t2_name_diffs)

{'George S. McGovern', 'Bob Dole'}


In [58]:
candidates_party_df.loc[(candidates_party_df['name']=='Bob Dole'), 'name'] = 'Robert Dole'

In [59]:
candidates_df.loc[(candidates_df['name_first']=='George')&(candidates_df['name_last']=='McGovern'), 'name'] = 'George S. McGovern'
candidates_df.loc[(candidates_df['name_first']=='George')&(candidates_df['name_last']=='McGovern'), 'name_middle'] = 'S.'

In [60]:
# Should be empty set now
t1_names = set(candidates_party_df['name'].unique())
t1_t2_name_diffs = t1_names.difference(set(candidates_df['name'].unique()))
print(f"Q: Are the candidate names derived from Table 2 and Table 1 spelled exactly the same? A: {len(t1_t2_name_diffs)==0}")

Q: Are the candidate names derived from Table 2 and Table 1 spelled exactly the same? A: True


#### **3.2.4 Construct Final Candidate Table**
Final Candidate table is created by left-joining the Candidates Party table to the Candidates State table, and then creating a new column, 'primary_key', that uniquely identifies each Candidate (i.e. row) of the table.

#### Join Candidate State DF and Candidate Party DF

In [61]:
candidates_df = candidates_df.merge(candidates_party_df, how='left', on='name', copy=False, validate='1:1')

In [62]:
candidates_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59 entries, 0 to 58
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         59 non-null     object
 1   name_first   59 non-null     object
 2   name_middle  35 non-null     object
 3   name_last    59 non-null     object
 4   name_suffix  2 non-null      object
 5   state        58 non-null     object
 6   state_2      2 non-null      object
 7   party        45 non-null     object
 8   party_2      2 non-null      object
dtypes: object(9)
memory usage: 4.6+ KB


#### Add Primary Key Column

In [63]:
candidates_df.reset_index(inplace=True)
candidates_df.rename({'index': 'candidate_id'}, axis=1, inplace=True)
candidates_df['candidate_id'] = candidates_df['candidate_id'].map(lambda x: x+1)

#### Replace NaN with None prior to facilitate Database Write

In [64]:
candidates_df = candidates_df.applymap(lambda x: None if pd.isnull(x) and x is not None else x)

#### **3.2.5 Validate Final Candidate Table**
Make sure that no candidate, state, or party data has been lost along the way.

In [65]:
candidates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   candidate_id  59 non-null     int64 
 1   name          59 non-null     object
 2   name_first    59 non-null     object
 3   name_middle   35 non-null     object
 4   name_last     59 non-null     object
 5   name_suffix   2 non-null      object
 6   state         58 non-null     object
 7   state_2       2 non-null      object
 8   party         45 non-null     object
 9   party_2       2 non-null      object
dtypes: int64(1), object(9)
memory usage: 4.7+ KB


In [66]:
candidates_df

,candidate_id,name,name_first,name_middle,name_last,name_suffix,state,state_2,party,party_2
0,1,Grover Cleveland,Grover,None,Cleveland,None,New York,None,D,None
1,2,Benjamin Harrison,Benjamin,None,Harrison,None,Indiana,None,R,None
2,3,James B. Weaver,James,B.,Weaver,None,Iowa,None,None,None
3,4,William McKinley,William,None,McKinley,None,Ohio,None,R,None
4,5,William J. Bryan,William,J.,Bryan,None,Nebraska,None,D,P
5,6,Theodore Roosevelt,Theodore,None,Roosevelt,None,New York,None,R,P
6,7,Alton B. Parker,Alton,B.,Parker,None,New York,None,D,None
7,8,William H. Taft,William,H.,Taft,None,Ohio,None,R,None
8,9,Woodrow Wilson,Woodrow,None,Wilson,None,New Jersey,None,D,None
9,10,Charles E. Hughes,Charles,E.,Hughes,None,New York,None,R,None


In [67]:
print(f"Q: Does each row correspond to a unique candidate name? A: {len(candidates_df) == len(candidates_df.drop_duplicates(subset='name'))}")

Q: Does each row correspond to a unique candidate name? A: True


In [68]:
num_final_candidates = len(candidates_df)
print(f"Q: Is the Final # of Candidates, {num_final_candidates}, equal to the # of Candidates from Table 2? A: {num_final_candidates==num_candidates}")

Q: Is the Final # of Candidates, 59, equal to the # of Candidates from Table 2? A: True


In [69]:
final_state_counts = candidates_df['state'].value_counts(dropna=False)
print(f"Q: Is the Final Distribution of Primary States equal to the Distribution of Primary States from Table 2? A: {(state_counts==final_state_counts).all()}")

Q: Is the Final Distribution of Primary States equal to the Distribution of Primary States from Table 2? A: True


In [70]:
final_state_2_counts = candidates_df['state_2'].value_counts(dropna=False)
print(f"Q: Is the Final Distribution of Secondary States equal to the Distribution of Secondary States from Table 2? A: {(state_2_counts==final_state_2_counts).all()}")

Q: Is the Final Distribution of Secondary States equal to the Distribution of Secondary States from Table 2? A: True


In [71]:
final_party_counts = candidates_df['party'].value_counts()
print(f"Q: Is the Final Distribution of Primary Party equal to the Distribution of Primary Party from Table 1? A: {(party_counts==final_party_counts).all()}")

Q: Is the Final Distribution of Primary Party equal to the Distribution of Primary Party from Table 1? A: True


In [72]:
final_party_2_counts = candidates_df['party_2'].value_counts()
print(f"Q: Is the Final Distribution of Secondary Party equal to the Distribution of Secondary Party from Table 1? A: {(party_2_counts==final_party_2_counts).all()}")

Q: Is the Final Distribution of Secondary Party equal to the Distribution of Secondary Party from Table 1? A: True


### 3.3 Transform and Validate State Data
Build state dataframe containing one row for each state, in alphabetical order, with the following information for each state:
- 'region' and 'division' corresponding to Census Bureau-designated regions and divisions, respectively
- 'statens' and 'geoid' corresponding to state ANSI code and FIPS code, respectively, which can be used to join to other datasets, e.g. Census population data, etc.
- 'state_usps' and 'state' corresponding to the state abbreviation and state name, respectively
- 'area_land' and 'area_water' corresponding to the land area and water area of the state, in m^2
- 'latitude' and 'longitude' corresponding to the latitude and longitude of the state's approximate centroid, in decimal degrees

**State names in 'state' column should always be unique and not null, so I've decided to use the state name as the primary key for the state dimension table, as an integer id column is unnecessary and would require an extra join to the votes fact table for many common reporting tasks.**

#### **3.3.1 Extract, Validate ad Transform State Data**

#### Build State DataFrame

In [73]:
state_df = pd.DataFrame(sorted(us_state_names), columns=['state'])
state_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   state   51 non-null     object
dtypes: object(1)
memory usage: 536.0+ bytes


In [74]:
# state_df is derived from a set, so all name values must be unique
num_states = len(state_df)
print(f"Unique number of States = {num_states}")

Unique number of States = 51


In [75]:
state_df.head()

,state
0,Alabama
1,Alaska
2,Arizona
3,Arkansas
4,California


#### Join to GeoPandas DataFrame with State geographic info
Remove the following columns from the GeoPandas DataFrame prior to joining:
- 'STATEFP', as it is always the same as 'GEOID' column
- 'LSAD', as it is not relevant and always equal to 00
- 'MTFCC', as it is not relevant and always equal to G4000
- 'FUNCSTAT', as it is not relevant and always equal to A

In [76]:
usa.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   REGION    56 non-null     object  
 1   DIVISION  56 non-null     object  
 2   STATEFP   56 non-null     object  
 3   STATENS   56 non-null     object  
 4   GEOID     56 non-null     object  
 5   STUSPS    56 non-null     object  
 6   NAME      56 non-null     object  
 7   LSAD      56 non-null     object  
 8   MTFCC     56 non-null     object  
 9   FUNCSTAT  56 non-null     object  
 10  ALAND     56 non-null     int64   
 11  AWATER    56 non-null     int64   
 12  INTPTLAT  56 non-null     object  
 13  INTPTLON  56 non-null     object  
 14  geometry  56 non-null     geometry
dtypes: geometry(1), int64(2), object(12)
memory usage: 6.7+ KB


In [77]:
usa.head()

,REGION,DIVISION,STATEFP,STATENS,GEOID,STUSPS,NAME,LSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,3,5,54,01779805,54,WV,West Virginia,00,G4000,A,62266231560,489271086,+38.6472854,-080.6183274,"POLYGON ((-81.74725 39.09538, -81.74635 39.096..."
1,3,5,12,00294478,12,FL,Florida,00,G4000,A,138947364717,31362872853,+28.4574302,-082.4091477,"MULTIPOLYGON (((-86.38865 30.99418, -86.38385 ..."
2,2,3,17,01779784,17,IL,Illinois,00,G4000,A,143779863817,6215723896,+40.1028754,-089.1526108,"POLYGON ((-91.18529 40.63780, -91.17510 40.643..."
3,2,4,27,00662849,27,MN,Minnesota,00,G4000,A,206230065476,18942261495,+46.3159573,-094.1996043,"POLYGON ((-96.78438 46.63050, -96.78434 46.630..."
4,3,5,24,01714934,24,MD,Maryland,00,G4000,A,25151726296,6979340970,+38.9466584,-076.6744939,"POLYGON ((-77.45881 39.22027, -77.45866 39.220..."


In [78]:
usa_cols_to_drop = ['STATEFP', 'LSAD', 'MTFCC', 'FUNCSTAT', 'geometry']
state_map_df = usa.drop(usa_cols_to_drop, axis=1, inplace=False)

In [79]:
state_map_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   REGION    56 non-null     object
 1   DIVISION  56 non-null     object
 2   STATENS   56 non-null     object
 3   GEOID     56 non-null     object
 4   STUSPS    56 non-null     object
 5   NAME      56 non-null     object
 6   ALAND     56 non-null     int64 
 7   AWATER    56 non-null     int64 
 8   INTPTLAT  56 non-null     object
 9   INTPTLON  56 non-null     object
dtypes: int64(2), object(8)
memory usage: 4.5+ KB


#### **3.3.2 Construct Final State Table**
Final State table is created using an inner join to the State GeoPandas DataFrame

In [80]:
state_df = state_df.merge(state_map_df, how='inner', left_on='state', right_on='NAME', validate='1:1')

In [81]:
state_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 0 to 50
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   state     51 non-null     object
 1   REGION    51 non-null     object
 2   DIVISION  51 non-null     object
 3   STATENS   51 non-null     object
 4   GEOID     51 non-null     object
 5   STUSPS    51 non-null     object
 6   NAME      51 non-null     object
 7   ALAND     51 non-null     int64 
 8   AWATER    51 non-null     int64 
 9   INTPTLAT  51 non-null     object
 10  INTPTLON  51 non-null     object
dtypes: int64(2), object(9)
memory usage: 4.8+ KB


#### Update Column Names

In [82]:
state_column_names = {'REGION': 'region', 'DIVISION': 'division', 'STATENS': 'statens', 'GEOID': 'geoid', 'STUSPS': 'state_usps', \
               'ALAND': 'area_land', 'AWATER': 'area_water', 'INTPTLAT': 'latitude', 'INTPTLON': 'longitude'}
state_df.drop(['NAME'], axis=1, inplace=True)
state_df.rename(state_column_names, axis=1, inplace=True)

#### Convert Appropriate Columns to Numeric Data Types
Convert Region, Division, Latitude and Longitude to numeric data types so they're formatted correctly for Postgres

In [83]:
state_df = state_df.astype({'region': 'int', 'division': 'int', 'latitude': 'float', 'longitude': 'float'}, copy=False)

In [84]:
state_df = state_df.iloc[:, [0, 5, 1, 2, 3, 4, 6, 7, 8, 9]]
state_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 0 to 50
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   state       51 non-null     object 
 1   state_usps  51 non-null     object 
 2   region      51 non-null     int64  
 3   division    51 non-null     int64  
 4   statens     51 non-null     object 
 5   geoid       51 non-null     object 
 6   area_land   51 non-null     int64  
 7   area_water  51 non-null     int64  
 8   latitude    51 non-null     float64
 9   longitude   51 non-null     float64
dtypes: float64(2), int64(4), object(4)
memory usage: 4.4+ KB


#### **3.3.3 Validate Final State Table**
Make sure that no state data has been lost along the way.

In [85]:
state_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 0 to 50
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   state       51 non-null     object 
 1   state_usps  51 non-null     object 
 2   region      51 non-null     int64  
 3   division    51 non-null     int64  
 4   statens     51 non-null     object 
 5   geoid       51 non-null     object 
 6   area_land   51 non-null     int64  
 7   area_water  51 non-null     int64  
 8   latitude    51 non-null     float64
 9   longitude   51 non-null     float64
dtypes: float64(2), int64(4), object(4)
memory usage: 4.4+ KB


In [86]:
state_df

,state,state_usps,region,division,statens,geoid,area_land,area_water,latitude,longitude
0,Alabama,AL,3,6,01779775,01,131174192284,4593183334,32.739632,-86.843459
1,Alaska,AK,4,9,01785533,02,1478927050067,245394222619,63.347356,-152.839733
2,Arizona,AZ,4,8,01779777,04,294360282618,859561204,34.203936,-111.606357
3,Arkansas,AR,3,7,00068085,05,134776580080,2956395922,34.895526,-92.444626
4,California,CA,4,9,01779778,06,403660088482,20305454540,37.155177,-119.543418
5,Colorado,CO,4,8,01779779,08,268419875371,1184637800,38.993848,-105.508317
6,Connecticut,CT,1,1,01779780,09,12542497381,1815617293,41.579864,-72.746657
7,Delaware,DE,3,5,01779781,10,5046620081,1399291164,38.998566,-75.441644
8,District of Columbia,DC,3,5,01702382,11,158340389,18687196,38.904247,-77.016517
9,Florida,FL,3,5,00294478,12,138947364717,31362872853,28.457430,-82.409148


In [87]:
print(f"Q: Does each row correspond to a unique state name? A: {len(state_df) == len(state_df.drop_duplicates(subset='state'))}")

Q: Does each row correspond to a unique state name? A: True


In [88]:
num_final_states = len(state_df)
print(f"Q: Is the Final # of States, {num_final_states}, equal to the Initial # of States? A: {num_final_states==num_states}")

Q: Is the Final # of States, 51, equal to the Initial # of States? A: True


### 3.4 Transform and Validate Electoral Votes Data

#### **3.4.1 Extract, Validate and Transform Electoral Votes Data**
This section bring it all together by joining the Final Candidate and State DataFrames to the Electoral Vote DataFrame to exchange the Candidate and State names for their primary keys, thus producing the Final Electoral Vote Fact Table, which can then be written to Postgres.

#### Validate the # of States Parsed for Each Election Year
Verify that the # of States that voted for President each year makes sense. I've confirmed that the state counts by year are consistent with when each state was added to the Union from 1892 to present, plus when Washington DC was allowed to vote (1964). See [this link](https://en.wikipedia.org/wiki/List_of_U.S._states_by_date_of_admission_to_the_Union) for details on when each state joined the Union and was able to cast electoral votes for President. Also, collect this data for validating the Final Votes by Year Table at the end of this section.

In [89]:
year_state_count = {}
for ind, pyr in enumerate(parsed_election_years):
    year_state_count[pyr['year']] = len(pyr['t2']['votes_by_state'])
initial_year_state_count = pd.Series(year_state_count, name='state_count')
print("Election Year, # of States w/ Vote Data Including State Totals")
initial_year_state_count

Election Year, # of States w/ Vote Data Including State Totals


1892    45
1896    46
1900    46
1904    46
1908    47
1912    49
1916    49
1920    49
1924    49
1928    49
1932    49
1936    49
1940    49
1944    49
1948    49
1952    49
1956    49
1960    51
1964    52
1968    52
1972    52
1976    52
1980    52
1984    52
1988    52
1992    52
1996    52
2000    52
2004    52
2008    52
2012    52
2016    52
2020    52
Name: state_count, dtype: int64

In [90]:
t2_votes_df = pd.json_normalize(parsed_election_years, ['t2', 'votes_by_state'], ['year'])

In [91]:
t2_votes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1649 entries, 0 to 1648
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   state                  1649 non-null   object 
 1   total_electoral_votes  1649 non-null   int64  
 2   1                      1649 non-null   int64  
 3   2                      1649 non-null   int64  
 4   3                      604 non-null    float64
 5   4                      52 non-null     float64
 6   year                   1649 non-null   object 
dtypes: float64(2), int64(3), object(2)
memory usage: 90.3+ KB


In [92]:
t2_votes_df.head(10)

,state,total_electoral_votes,1,2,3,4,year
0,Alabama,11,11,0,0.0,NaN,1892
1,Arkansas,8,8,0,0.0,NaN,1892
2,California,9,8,1,0.0,NaN,1892
3,Colorado,4,0,0,4.0,NaN,1892
4,Connecticut,6,6,0,0.0,NaN,1892
5,Delaware,3,3,0,0.0,NaN,1892
6,Florida,4,4,0,0.0,NaN,1892
7,Georgia,13,13,0,0.0,NaN,1892
8,Idaho,3,0,0,3.0,NaN,1892
9,Illinois,24,24,0,0.0,NaN,1892


#### Add 2016 Rows for "Other" Candidates Electoral Votes

In [93]:
other_vote_inds = t2_votes_df[t2_votes_df['year']==2016].index

In [94]:
# Set Existing Vote data for "Other" Candidate columns with 'col_ind' = 2, 4
# to zero, and initialize columns for 'col_ind' = 5, 6, 7 with a value of zero
# Then, reorder columns so year is last
other_vote_col_inds = other_candidates_df['col_ind'].tolist()
t2_votes_df.loc[other_vote_inds, other_vote_col_inds] = 0
t2_votes_df = t2_votes_df.iloc[:, [0, 1, 2, 3, 4, 5, 7, 8, 9, 6]]
t2_votes_df.loc[other_vote_inds, :]

,state,total_electoral_votes,1,2,3,4,5,6,7,year
1545,Alabama,9,9,0,0.0,0.0,0.0,0.0,0.0,2016
1546,Alaska,3,3,0,0.0,0.0,0.0,0.0,0.0,2016
1547,Arizona,11,11,0,0.0,0.0,0.0,0.0,0.0,2016
1548,Arkansas,6,6,0,0.0,0.0,0.0,0.0,0.0,2016
1549,California,55,0,0,55.0,0.0,0.0,0.0,0.0,2016
1550,Colorado,9,0,0,9.0,0.0,0.0,0.0,0.0,2016
1551,Connecticut,7,0,0,7.0,0.0,0.0,0.0,0.0,2016
1552,Delaware,3,0,0,3.0,0.0,0.0,0.0,0.0,2016
1553,District of Columbia,3,0,0,3.0,0.0,0.0,0.0,0.0,2016
1554,Florida,29,29,0,0.0,0.0,0.0,0.0,0.0,2016


In [95]:
other_candidates_df

,president_candidate_name,col_ind,president_candidate_state,year
76.0,Bernie Sanders,2,Vermont,2016
74.0,Ron Paul,4,Texas,2016
74.1,John Kasich,5,Ohio,2016
74.2,Colin Powell,6,None,2016
76.1,Faith Spotted Eagle,7,South Dakota,2016


In [96]:
# Collected from Notes section here: https://www.archives.gov/electoral-college/2016
other_votes_df = pd.DataFrame({'state': ["Hawaii", "Texas", "Texas", "Washington", "Washington"],
                               'col_ind': other_vote_col_inds,
                               'votes': [1, 1, 1, 3, 1]})
other_votes_df

,state,col_ind,votes
0,Hawaii,2,1
1,Texas,4,1
2,Texas,5,1
3,Washington,6,3
4,Washington,7,1


In [97]:
# Update t2_votes_df with data from other_votes_df
for tup in other_votes_df.itertuples(index=False):
    t2_votes_df.loc[(t2_votes_df.index.isin(other_vote_inds))&(t2_votes_df['state']==tup.state), tup.col_ind] = tup.votes

In [98]:
# Update Row with Totals for newly added "Other" Candidates' vote data
# Last Index identifies Totals row
t2_votes_df.loc[other_vote_inds[-1], other_vote_col_inds] = t2_votes_df.loc[other_vote_inds[:-1], other_vote_col_inds].sum(axis=0)

In [99]:
t2_votes_df.loc[other_vote_inds, :]

,state,total_electoral_votes,1,2,3,4,5,6,7,year
1545,Alabama,9,9,0,0.0,0.0,0.0,0.0,0.0,2016
1546,Alaska,3,3,0,0.0,0.0,0.0,0.0,0.0,2016
1547,Arizona,11,11,0,0.0,0.0,0.0,0.0,0.0,2016
1548,Arkansas,6,6,0,0.0,0.0,0.0,0.0,0.0,2016
1549,California,55,0,0,55.0,0.0,0.0,0.0,0.0,2016
1550,Colorado,9,0,0,9.0,0.0,0.0,0.0,0.0,2016
1551,Connecticut,7,0,0,7.0,0.0,0.0,0.0,0.0,2016
1552,Delaware,3,0,0,3.0,0.0,0.0,0.0,0.0,2016
1553,District of Columbia,3,0,0,3.0,0.0,0.0,0.0,0.0,2016
1554,Florida,29,29,0,0.0,0.0,0.0,0.0,0.0,2016


In [100]:
t2_votes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1649 entries, 0 to 1648
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   state                  1649 non-null   object 
 1   total_electoral_votes  1649 non-null   int64  
 2   1                      1649 non-null   int64  
 3   2                      1649 non-null   int64  
 4   3                      604 non-null    float64
 5   4                      52 non-null     float64
 6   5                      52 non-null     float64
 7   6                      52 non-null     float64
 8   7                      52 non-null     float64
 9   year                   1649 non-null   object 
dtypes: float64(5), int64(3), object(2)
memory usage: 129.0+ KB


#### Validate State Electoral Votes
Make sure that the total number of Electoral Votes for each state is equal to the sum of the Electoral Votes across all candidates, for each election year. All rows check out, except
- **"District of Columbia"**, with a total of 3 but only 2 votes allocated
- **"Totals"**, with a total of 538 but only 537 votes allocated 

I've writting to the National Archive about this issue, so they can fix it. In the meantime, I'm going to manually fix it in the next subsection

In [101]:
vote_row_sum = t2_votes_df.loc[:, [1, 2, 3, 4, 5, 6, 7]].sum(axis=1)
t2_votes_df[vote_row_sum!=t2_votes_df['total_electoral_votes']]

,state,total_electoral_votes,1,2,3,4,5,6,7,year
1345,District of Columbia,3,0,2,NaN,NaN,NaN,NaN,NaN,2000
1388,Totals,538,271,266,NaN,NaN,NaN,NaN,NaN,2000


#### Fix 2000 Electoral Vote Error for DC and Totals

In [102]:
t2_votes_df.loc[(t2_votes_df['state']=="District of Columbia") & (t2_votes_df['year']==2000), 2] = \
    t2_votes_df.loc[(t2_votes_df['state']=="District of Columbia") & (t2_votes_df['year']==2000), 'total_electoral_votes']
t2_votes_df.loc[(t2_votes_df['state']=="Totals") & (t2_votes_df['year']==2000), 2] = \
    t2_votes_df.loc[(t2_votes_df['state']!="Totals") & (t2_votes_df['year']==2000), 2].sum(axis=0)

In [103]:
# The two erroneous rows identified in the previous section should be gone now
vote_row_sum = t2_votes_df.loc[:, [1, 2, 3, 4, 5, 6, 7]].sum(axis=1)
t2_votes_df[vote_row_sum!=t2_votes_df['total_electoral_votes']]

,state,total_electoral_votes,1,2,3,4,5,6,7,year


#### Build Initial Electoral Votes DataFrame
Un-pivot the 'col_ind' column so that there is a single column that contains the index corresponding to each Candidate that received at least one Electoral Vote in a given Election Year. This format is required so that the Final Candidate DataFrame can be joined to the Final Electoral Vote DataFrame via the `t2_states_df` DataFrame created in Section 3.2.1, as documented in the following sections.

In [104]:
votes_df = pd.melt(t2_votes_df, id_vars=['year', 'state', 'total_electoral_votes'], value_vars=[1, 2, 3, 4, 5, 6, 7], \
              var_name='col_ind', value_name='president_electoral_votes')

In [105]:
votes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11543 entries, 0 to 11542
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   year                       11543 non-null  object 
 1   state                      11543 non-null  object 
 2   total_electoral_votes      11543 non-null  int64  
 3   col_ind                    11543 non-null  object 
 4   president_electoral_votes  4110 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 451.0+ KB


In [106]:
num_initial_vote_rows = len(votes_df)
print(f"Initial # of rows containing vote data = {num_initial_vote_rows}")

Initial # of rows containing vote data = 11543


#### Remove rows with Null Vote Data
The null rows are a result of un-pivoting the origin electoral votes DataFrame, because year 2016 had 4 candidates with votes so a 'col_ind'=4 column was needed in that format, plus only a handful of elections had 3 candidates, so most of the rows with 'col_ind'=3 have no vote data associated with them. Also convert vote column to integer once nulls have been removed (only float type is allowed when nulls are present).

In [107]:
votes_df.dropna(inplace=True)
votes_df = votes_df.astype({'president_electoral_votes': 'int'}, copy=False)
num_clean_vote_rows = len(votes_df)
print(f"# of cleaned rows containing vote data = {num_clean_vote_rows}")

# of cleaned rows containing vote data = 4110


In [108]:
num_unique_year_colind = len(votes_df.drop_duplicates(subset=['year', 'col_ind']))

In [109]:
votes_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4110 entries, 0 to 11490
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   year                       4110 non-null   object
 1   state                      4110 non-null   object
 2   total_electoral_votes      4110 non-null   int64 
 3   col_ind                    4110 non-null   object
 4   president_electoral_votes  4110 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 192.7+ KB


#### **3.4.2 Construct Final Votes by Year Table**

#### Correct Candidate Name mismatches between t2_state_df and Final Candidate Table
Full name values need to be consistent between candidates_df and t2_state_df so that the join to the Votes by Year works correctly in Section 3.4.2.

In [110]:
t2_states_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   president_candidate_name   82 non-null     object
 1   col_ind                    82 non-null     int64 
 2   president_candidate_state  81 non-null     object
 3   year                       82 non-null     object
dtypes: int64(1), object(3)
memory usage: 2.7+ KB


In [111]:
vote_names = set(t2_states_df['president_candidate_name'].unique())
vote_cand_name_diffs = vote_names.difference(set(candidates_df['name'].unique()))
print(vote_cand_name_diffs)

{'Donald Trump', 'George McGovern'}


In [112]:
t2_states_df.loc[(t2_states_df['president_candidate_name']=='Donald Trump'), 'president_candidate_name'] = \
candidates_df.loc[(candidates_df['name_first']=='Donald')&(candidates_df['name_last']=='Trump'), 'name'].values[0]

In [113]:
t2_states_df.loc[(t2_states_df['president_candidate_name']=='George McGovern'), 'president_candidate_name'] = \
candidates_df.loc[(candidates_df['name_first']=='George')&(candidates_df['name_last']=='McGovern'), 'name'].values[0]

In [114]:
# Should be empty set now
vote_names = set(t2_states_df['president_candidate_name'].unique())
vote_cand_name_diffs = vote_names.difference(set(candidates_df['name'].unique()))
print(f"Q: Are the candidate names derived from Table 2 and Table 1 spelled exactly the same? A: {len(vote_cand_name_diffs)==0}")

Q: Are the candidate names derived from Table 2 and Table 1 spelled exactly the same? A: True


#### Join Vote DataFrame to the t2_states_df
This join connects the transformed votes data in votes_df to the candidate-to-column id data in `t2_states_df` using the 'year' and 'col_ind' columns.

In [115]:
votes_df = votes_df.merge(t2_states_df, how='inner', on=['year', 'col_ind'], validate='m:1')

In [116]:
votes_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4110 entries, 0 to 4109
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   year                       4110 non-null   object
 1   state                      4110 non-null   object
 2   total_electoral_votes      4110 non-null   int64 
 3   col_ind                    4110 non-null   object
 4   president_electoral_votes  4110 non-null   int64 
 5   president_candidate_name   4110 non-null   object
 6   president_candidate_state  4058 non-null   object
dtypes: int64(2), object(5)
memory usage: 256.9+ KB


#### Join Vote DataFrame to the Final Candidate DataFrame
This inner join connects the transformed, merged votes data in `votes_df` to the final candidate data in `candidates_df` using the 'name'/'president_candidate_name' column. Only the 'name' and 'candidate_id' columns are needed from `candidates_df`, the former to join on and the latter to include in `votes_df`.

In [117]:
join_candidates_df = candidates_df[['candidate_id', 'name']].copy()
votes_df = votes_df.merge(join_candidates_df, how='inner', left_on='president_candidate_name', right_on='name', validate='m:1')
votes_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4110 entries, 0 to 4109
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   year                       4110 non-null   object
 1   state                      4110 non-null   object
 2   total_electoral_votes      4110 non-null   int64 
 3   col_ind                    4110 non-null   object
 4   president_electoral_votes  4110 non-null   int64 
 5   president_candidate_name   4110 non-null   object
 6   president_candidate_state  4058 non-null   object
 7   candidate_id               4110 non-null   int64 
 8   name                       4110 non-null   object
dtypes: int64(3), object(6)
memory usage: 321.1+ KB


#### Join Votes DataFrame to the Final State DataFrame
This left join connects the transformed, merged votes data in `votes_df` to the final state data in `states_df` using the 'state' column. A left join is required because of the state value of 'Totals', in `votes_df`, which isn't an actual state, of course, but instead contains the overall electoral vote tally for each election year and candidate. Only the 'state' column is needed from `state_df`, and a separate column, 'is_total', is added to identify rows that contain vote totals, as the 'state' column will be a foreign key to the state dimension table so can't contain the value 'Totals'.

**Note: this step can easily be implemented without the join, but I prefer to follow this more formal pattern to validate the votes data, and in case I want to make other changes later**

In [118]:
join_states_df = state_df[['state']].copy()
votes_df = votes_df.merge(join_states_df, how='left', on='state', indicator=True, validate='m:1')
votes_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4110 entries, 0 to 4109
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   year                       4110 non-null   object  
 1   state                      4110 non-null   object  
 2   total_electoral_votes      4110 non-null   int64   
 3   col_ind                    4110 non-null   object  
 4   president_electoral_votes  4110 non-null   int64   
 5   president_candidate_name   4110 non-null   object  
 6   president_candidate_state  4058 non-null   object  
 7   candidate_id               4110 non-null   int64   
 8   name                       4110 non-null   object  
 9   _merge                     4110 non-null   category
dtypes: category(1), int64(3), object(6)
memory usage: 325.2+ KB


In [119]:
# '_merge' = 'both' when an actual state name is present
# '_merge' = 'left_only' when 'state' = 'Other'
votes_df['_merge'].value_counts()

both          4028
left_only       82
right_only       0
Name: _merge, dtype: int64

#### Remove state='Totals' Values, Add 'is_total' Column

In [120]:
votes_df.loc[votes_df['_merge']=='left_only', 'state'] = None
votes_df['is_total'] = votes_df['_merge'].map(lambda x: True if x == 'left_only' else False)
votes_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4110 entries, 0 to 4109
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   year                       4110 non-null   object  
 1   state                      4028 non-null   object  
 2   total_electoral_votes      4110 non-null   int64   
 3   col_ind                    4110 non-null   object  
 4   president_electoral_votes  4110 non-null   int64   
 5   president_candidate_name   4110 non-null   object  
 6   president_candidate_state  4058 non-null   object  
 7   candidate_id               4110 non-null   int64   
 8   name                       4110 non-null   object  
 9   _merge                     4110 non-null   category
 10  is_total                   4110 non-null   object  
dtypes: category(1), int64(3), object(7)
memory usage: 357.3+ KB


#### Remove Unnecessary Columns
Only the following columns are included in the Final Votes by Year table: 'year', 'state', 'candidate_id', 'total_electoral_votes', 'president_electoral_votes'. The 'col_ind' column is kept at this stage as an approximation for the electoral vote rank of each presidential candidate. It is discarded later once a true rank column is calculated to replace it, but it is needed to uniquely identify each Candidate by name due to 'president_candidate_name'='Other' value for 'year'=2016.

In [121]:
vote_cols_to_remove = ['president_candidate_name', 'president_candidate_state', 'name', '_merge']
votes_df.drop(vote_cols_to_remove, axis=1, inplace=True)
votes_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4110 entries, 0 to 4109
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   year                       4110 non-null   object
 1   state                      4028 non-null   object
 2   total_electoral_votes      4110 non-null   int64 
 3   col_ind                    4110 non-null   object
 4   president_electoral_votes  4110 non-null   int64 
 5   candidate_id               4110 non-null   int64 
 6   is_total                   4110 non-null   object
dtypes: int64(3), object(4)
memory usage: 256.9+ KB


#### Add Column with President Electoral Vote Rank
I've decided to add a column that defines the rank of each Presidential Candidate for each election year. I could keep the 'col_ind' column as it almost always serves this purpose, but almost isn't good enough. For example, it is incorrect for 2016 when there were two candidate names listed as 'Other', with 'col_ind' values of 2 and 4, even though the candidate with a 'col_ind' value of 3 received the second most electoral votes. Including a rank column at this stage in the data processing pipeline, rather than waiting to define it in a Data Mart level object, which I'd normally prefer, is helpful for validation purposes so that is ultimately why I decided on this approach.

**Note: I'm using a dense ranking algorithm, which technically could be a problem if two or more "Candidates" with the same name value (e.g. 'Other' for 2016) receive exactly the same # of electoral votes; however, this is not an issue in practice, and will hopefully motivate me to circle back and "unpack" the 'Other' candidates with the actual names of each Candidate that received one or more electoral votes in 2016.**

In [122]:
# The vote totals for each year have NaN values for 'state_id'
votes_totals_df = votes_df.loc[votes_df['state'].isna(), ['year', 'candidate_id', 'col_ind', 'president_electoral_votes']].sort_values(by='year')
votes_totals_df['president_electoral_rank'] = votes_totals_df.groupby("year")["president_electoral_votes"].rank("dense", ascending=False).astype('int')
votes_totals_df.drop('president_electoral_votes', axis=1, inplace=True)

In [123]:
votes_df = votes_df.merge(votes_totals_df, how='inner', on=['year', 'candidate_id', 'col_ind'], validate='m:1')
votes_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4110 entries, 0 to 4109
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   year                       4110 non-null   object
 1   state                      4028 non-null   object
 2   total_electoral_votes      4110 non-null   int64 
 3   col_ind                    4110 non-null   object
 4   president_electoral_votes  4110 non-null   int64 
 5   candidate_id               4110 non-null   int64 
 6   is_total                   4110 non-null   object
 7   president_electoral_rank   4110 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 289.0+ KB


In [124]:
# Only 'year'=2016 has a mismatch between 'col_ind' and 'president_electoral_rank' values, as expected
# so it is now safe to remove the 'col_ind' column
votes_df.loc[votes_df['col_ind']!=votes_df['president_electoral_rank'], ['year', 'col_ind', 'president_electoral_rank']].drop_duplicates()

,year,col_ind,president_electoral_rank
3347,2016,2,4
3850,2016,3,2
3954,2016,5,4
4006,2016,6,3
4058,2016,7,4


In [125]:
votes_df.drop('col_ind', axis=1, inplace=True)

#### Convert Column Types to Numeric

In [126]:
# 'state_id' must remain float type due to presence of NaN values
votes_df = votes_df.astype({'year': 'int', 'is_total': 'bool'}, copy=False)
votes_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4110 entries, 0 to 4109
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   year                       4110 non-null   int64 
 1   state                      4028 non-null   object
 2   total_electoral_votes      4110 non-null   int64 
 3   president_electoral_votes  4110 non-null   int64 
 4   candidate_id               4110 non-null   int64 
 5   is_total                   4110 non-null   bool  
 6   president_electoral_rank   4110 non-null   int64 
dtypes: bool(1), int64(5), object(1)
memory usage: 228.8+ KB


#### Update Column Order

In [127]:
votes_df = votes_df.iloc[:, [0, 1, 5, 4, 2, 3, 6]]
votes_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4110 entries, 0 to 4109
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   year                       4110 non-null   int64 
 1   state                      4028 non-null   object
 2   is_total                   4110 non-null   bool  
 3   candidate_id               4110 non-null   int64 
 4   total_electoral_votes      4110 non-null   int64 
 5   president_electoral_votes  4110 non-null   int64 
 6   president_electoral_rank   4110 non-null   int64 
dtypes: bool(1), int64(5), object(1)
memory usage: 228.8+ KB


#### Sort Rows by Year, State, Candidate

In [128]:
votes_df.sort_values(by=['year', 'state', 'is_total', 'candidate_id', 'president_electoral_rank'], inplace=True, ignore_index=True)
votes_df.head()

,year,state,is_total,candidate_id,total_electoral_votes,president_electoral_votes,president_electoral_rank
0,1892,Alabama,False,1,11,11,1
1,1892,Alabama,False,2,11,0,2
2,1892,Alabama,False,3,11,0,3
3,1892,Arkansas,False,1,8,8,1
4,1892,Arkansas,False,2,8,0,2


#### Add Primary Key Column
An integer identifier is best for a primary key in this case because one of the natural keys, 'state', contains null values. I may want to separate the total vote values for each election year into a separate table in the future so that I can better define a set of natural keys for the votes by state fact table...

In [129]:
votes_df.reset_index(inplace=True)
votes_df.rename({'index': 'votes_id'}, axis=1, inplace=True)
votes_df['votes_id'] = votes_df['votes_id'].map(lambda x: x+1)
votes_df.head()

,votes_id,year,state,is_total,candidate_id,total_electoral_votes,president_electoral_votes,president_electoral_rank
0,1,1892,Alabama,False,1,11,11,1
1,2,1892,Alabama,False,2,11,0,2
2,3,1892,Alabama,False,3,11,0,3
3,4,1892,Arkansas,False,1,8,8,1
4,5,1892,Arkansas,False,2,8,0,2


#### **3.4.3 Validate Final Votes by Year Table**
Make sure that no vote data has been lost along the way.

In [130]:
votes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4110 entries, 0 to 4109
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   votes_id                   4110 non-null   int64 
 1   year                       4110 non-null   int64 
 2   state                      4028 non-null   object
 3   is_total                   4110 non-null   bool  
 4   candidate_id               4110 non-null   int64 
 5   total_electoral_votes      4110 non-null   int64 
 6   president_electoral_votes  4110 non-null   int64 
 7   president_electoral_rank   4110 non-null   int64 
dtypes: bool(1), int64(6), object(1)
memory usage: 228.9+ KB


In [131]:
num_final_vote_rows = len(votes_df)
print(f"Q: Is the # of cleaned rows containing vote data, {num_clean_vote_rows}, equal to the final # of rows? A: {num_clean_vote_rows==num_final_vote_rows}")

Q: Is the # of cleaned rows containing vote data, 4110, equal to the final # of rows? A: True


In [132]:
# This count INCLUDES the state totals for each election year and candidate
final_year_state_count = votes_df[votes_df['president_electoral_rank']==1].groupby('year')['state'].size()
year_state_count = pd.merge(left=initial_year_state_count, right=final_year_state_count, left_index=True, right_index=True)
print(f"Q: Are the Initial and Final State Counts by Year Equivalent? A: {(year_state_count['state']==year_state_count['state_count']).all()}")

Q: Are the Initial and Final State Counts by Year Equivalent? A: True


In [133]:
# Check that the Total Votes values scraped from the National Archive website, and available in votes_df with the 'state_id' value equal to NaN, are
# equal to the vote total obtained by summing the electoral votes across all states, for each election year and candidate
states_votes_totals_df = votes_df.loc[votes_df['state'].notna(), ['year', 'candidate_id', 'president_electoral_rank', 'president_electoral_votes']]
states_votes_totals_df = states_votes_totals_df.groupby(['year', 'candidate_id', 'president_electoral_rank'])["president_electoral_votes"].sum().reset_index()
votes_totals_df = votes_df.loc[votes_df['state'].isna(), ['year', 'candidate_id', 'president_electoral_rank', 'president_electoral_votes']].reset_index(drop=True)
print("Q: Are the Vote Totals Scraped from the National Archive Website Equal to the Sum of the Votes across All States, for each Candidate and Election Year?")
print(f"A: {(votes_totals_df==states_votes_totals_df).all().all()}")

Q: Are the Vote Totals Scraped from the National Archive Website Equal to the Sum of the Votes across All States, for each Candidate and Election Year?
A: True


## 4. Write Election Data to Postgres Database

### 4.1 Define DB Connection, Schema, and Table Parameters

In [134]:
# Define parameters for database connection, schema name, table name, and column definitions
# Obfuscate password using getpass
db_config = {
    'user': 'postgres',
    'host': 'localhost',
    'port': '5432',
    'dbname': 'elections',
    'password': getpass.getpass()
}
# Data Warehouse schema
schema = 'dwh'
# Name of each table to create
table_names = ['state', 'candidate', 'votes']
# Define each column's name, type, and optionally constraint(s):
# one tuple per column + table constraint(s) at the end
# Column defs should be listed in the same order as table_names
# Any table with an FK reference must be defined after the
# table it is referencing, as shown below
table_column_defs = [
    [   #  State Dimension Table
        ('state', 'varchar', 'primary key'),
        ('state_usps', 'varchar(2)', 'not null'),
        ('region', 'smallint', 'not null'),
        ('division', 'smallint', 'not null'),
        ('statens', 'varchar', 'not null'),
        ('geoid', 'varchar', 'not null'),
        ('area_land', 'bigint', 'not null'),
        ('area_water', 'bigint', 'not null'),
        ('latitude', 'numeric', 'not null'),
        ('longitude', 'numeric', 'not null')
    ],
    [   # Candidate Dimension Table
        ('candidate_id', 'smallint', 'primary key'),
        ('name', 'varchar', 'not null'),
        ('name_first', 'varchar', 'not null'),
        ('name_middle', 'varchar'),
        ('name_last', 'varchar'),
        ('name_suffix', 'varchar'),
        ('state', 'varchar', f'REFERENCES {schema}.{table_names[0]}'),
        ('state_2', 'varchar', f'REFERENCES {schema}.{table_names[0]}'),
        ('party', 'varchar'),
        ('party_2', 'varchar')
    ],
    [   # Votes Fact Table
        ('votes_id', 'integer', 'primary key'),
        ('year', 'smallint', 'not null'),
        ('state', 'varchar', f'REFERENCES {schema}.{table_names[0]}'),
        ('is_total', 'boolean', 'not null'),
        ('candidate_id', 'smallint', 'not null', f'REFERENCES {schema}.{table_names[1]}'),
        ('total_electoral_votes', 'smallint', 'not null'),
        ('president_electoral_votes', 'smallint', 'not null'),
        ('president_electoral_rank', 'smallint', 'not null')
    ]
]

 ·········


### 4.2 Connect to Database via DBC Class
Create a Database Connection (DBC) instance for interacting with the provided postgres database (see db_tools.py for class definition). Note that the connection must be closed using the close flag on one of the instance methods, or by calling dbc.close_connection() explicitly.

In [135]:
dbc = DBC(db_config)

### 4.3 Write State, Candidate, and Votes data to Postgres Database

In [136]:
dfs = [state_df, candidates_df, votes_df]
create_tables_from_dfs(dbc, schema, table_names, table_column_defs, dfs, replace=True, close=True)

Creating Schema: dwh
Creating Table: dwh.state
Creating Table: dwh.candidate
Creating Table: dwh.votes
All Data Successfully Inserted!!!
